In [ ]:
import sys

sys.path.append("../")
from sgm.modules.lipreader.lightnining import ModelModule
import torch


def get_lightning_module(ckpt_path):
    modelmodule = ModelModule()
    modelmodule.model.load_state_dict(torch.load(ckpt_path, map_location=lambda storage, loc: storage))
    return modelmodule


ckpt_path = "/vol/paramonos2/projects/antoni/code/Personal/lipreader/models/vsr_trlrs3_base.max400.pth"
model = get_lightning_module(ckpt_path)
model.eval()

In [1]:
import sys

sys.path.append("../")
import numpy as np
import torch
import torchvision
from sgm.modules.lipreader.preparation.detectors.retinaface.video_process import VideoProcess
from sgm.modules.lipreader.datamodule.transforms import VideoTransform

video_transform = VideoTransform(subset="val", max_noise_level=250, center_crop_size=11)
video_process = VideoProcess(
    convert_gray=False, crop_width=96 // 8, crop_height=96 // 8, reference_size=(64, 64), target_size=(64, 64)
)
video_path = "/data2/Datasets/HDTF/cropped_videos_original/RD_Radio18_000.mp4"
video = torchvision.io.read_video(video_path, pts_unit="sec", output_format="TCHW")[0]
or_shape = video.shape[-2:]
video = torch.nn.functional.interpolate(video, size=(64, 64)).permute(0, 2, 3, 1).numpy()
print(video.shape)
landmarks = np.load("/data2/Datasets/HDTF/cropped_videos_original/RD_Radio18_000.npy")

/vol/paramonos2/projects/antoni/miniconda3/envs/svd/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(1825, 64, 64, 3)


In [14]:
pad = 10
xmouthpoints = (crop // 8)[48:67, 0]
ymouthpoints = (crop // 8)[48:67, 1]
maxx = np.max(xmouthpoints)
minx = np.min(xmouthpoints)
maxy = np.max(ymouthpoints)
miny = np.min(ymouthpoints)
print(maxx, minx, maxy, miny)
print(miny - pad, maxy + pad, minx - pad, maxx + pad)

4.0 3.0 5.0 4.0
-6.0 15.0 -7.0 14.0


In [3]:
_, crop = video_process.crop_patch(video, landmarks, return_bbox=True, threshold=30)

In [4]:
crop

array([[33, 45, 26, 38],
       [33, 45, 26, 38],
       [33, 45, 27, 39],
       ...,
       [33, 45, 26, 38],
       [33, 45, 26, 38],
       [33, 45, 26, 38]])

In [2]:
from sgm.data.data_utils import scale_landmarks

landmarks = scale_landmarks(landmarks, (or_shape[0], or_shape[1]), (64, 64))

In [7]:
96 // 8

12

In [15]:
video_proccessed = video_process(video, landmarks, True, threshold=30)
video_proccessed = torch.tensor(video_proccessed)
video_proccessed = video_proccessed.permute((0, 3, 1, 2))
video_proccessed, t = video_transform(video_proccessed)
t

tensor([0])

In [5]:
video_proccessed.shape, t

(torch.Size([1825, 1, 11, 11]), tensor([0]))

In [16]:
from PIL import Image

# Display first 10 frames video
for i in range(10):
    pil_image = (video_proccessed[i].permute(1, 2, 0).numpy() * 255).astype(np.uint8)

    display(Image.fromarray(pil_image.squeeze()), "L")

'L'

'L'

'L'

'L'

'L'

'L'

'L'

'L'

'L'

'L'

In [ ]:
model = model.to("cuda")
with torch.no_grad():
    print(model(video_proccessed[:250].cuda(), extract_position="conformer").shape)

In [ ]:
model = model.to("cuda")
duration = 15


with torch.no_grad():
    input = video_proccessed[: 25 * duration].cuda()

    pred_noise = model(input, t=t.cuda())
    print(pred_noise)

    pred = model(input, t=None)
    print(pred)

In [ ]:
import torchaudio


def compute_word_level_distance(seq1, seq2):
    return torchaudio.functional.edit_distance(seq1.lower().split(), seq2.lower().split())